## 합성곱/풀링 계층 구현

### CNN에 흐르는 데이터는 4차원: 4차원 배열

In [1]:
import numpy as np

x = np.random.rand(10, 1, 28, 28) # 무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [2]:
x[0].shape

(1, 28, 28)

In [3]:
x[1].shape

(1, 28, 28)

In [4]:
x[0, 0]

array([[0.02273987, 0.90956469, 0.01861745, 0.22701893, 0.07923403,
        0.08946858, 0.48028666, 0.99531842, 0.19329916, 0.97085939,
        0.5499775 , 0.58194466, 0.6703877 , 0.33629993, 0.70605742,
        0.56850395, 0.89665789, 0.03363918, 0.75633569, 0.81786297,
        0.39379395, 0.43080974, 0.7990908 , 0.59864367, 0.36269674,
        0.95280384, 0.45531776, 0.67089866],
       [0.1659332 , 0.78614575, 0.0343028 , 0.54468551, 0.38426693,
        0.69917864, 0.66222517, 0.92454983, 0.98075824, 0.50938092,
        0.41050206, 0.49284576, 0.64820609, 0.2002884 , 0.13062983,
        0.48279906, 0.38651683, 0.46221502, 0.47113925, 0.78747573,
        0.13130543, 0.5444071 , 0.12121571, 0.19442746, 0.9224413 ,
        0.29513868, 0.88136794, 0.31663437],
       [0.63505494, 0.59618546, 0.86973985, 0.87006516, 0.99303284,
        0.83048665, 0.57459052, 0.03308915, 0.57878565, 0.81758845,
        0.10554143, 0.92871008, 0.7679634 , 0.6524867 , 0.13353229,
        0.17039431, 0.4619

### im2col 사용해서 합성곱 계층 구현
- im2col(input_data, filter_h, filter_w, stride=1, pad=0)

In [5]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

In [6]:
x1 = np.random.rand(1, 3, 7, 7) # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [7]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [9]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0): # 가중치, 편향, 스트라이드, 패딩을 인수로 받음
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape # 필터개수, 채널, 필터높이, 필터너비
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T  # 필터 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

### 풀링 계층 구현

In [10]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        # 최댓값 (2)
        out = np,max(col, axis=1)
        
        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

## CNN 구현하기

- SimpleConvNet: Convolution-ReLU-Pooling-Affine-ReLU-Affine-Softmax 순

In [ ]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28)), 
                conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, 
                hidden_size=100, output_size=10, weight_init_std=0.01:
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_parma['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()